In [2]:
import json
import pandas as pd
from useful_functions import get_opsea_trans_by_collec, extract_asset_events, combine_json_files, get_traits_collection, get_nft_traits

### 0. Parameter Specification

In [3]:
data = "boredapeyachtclub, mutant-ape-yacht-club, bored-ape-kennel-club"

words = data.split(',')  # Returns a list
collection_list = ['' + word.strip() + '' for word in words]  # I split it at , and then append to the list
after = '2020-01-01 01:00:00'  # 1st of January 2020 01:00:00 am
before = '2023-12-01 01:00:00'  # 1st of December 2023 01:00:00 am
event_type = 'sale'  # Other types: all, cancel, redemption, transfer

### 1. Downloading transactions

In [ ]:
for collection in collection_list:
    get_opsea_trans_by_collec(collection, before, after, event_type, collection)

### 2. Combine the JSON files fo transactions into one

In [ ]:
# Combine the JSON files into one
combine_json_files(collection_list, 'combined_transactions')

### 3. Add to a dataframe

In [4]:
with open('Transaction_files/combined_transactions.json') as z:
    trans_3years = json.load(z)

In [5]:
# Extract list of transactions from transaction blocks and append to a list
list_of_transactions_clean = extract_asset_events(trans_3years)

In [6]:
df = pd.json_normalize(list_of_transactions_clean)
df.head()  # 28 columns

,event_type,order_hash,chain,protocol_address,closing_date,quantity,seller,buyer,transaction,event_timestamp,...,nft.opensea_url,nft.updated_at,nft.is_disabled,nft.is_nsfw,payment.quantity,payment.token_address,payment.decimals,payment.symbol,nft,payment
0,sale,,ethereum,,1701378215,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0xae7fca4f38a28fe3580d42d7a56190ff4b80dc65,0xb085fc8e1fa7518ced3c411a9184551138db03c75a53...,1701378215,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2024-01-13T02:50:25.797011,False,False,28250000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
1,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x46667400bc0225903e71d4f16f1625b239c72617a326...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2023-12-27T06:22:13.050808,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
2,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x9afef7dac35f070dc3976f0597b8c872f6b19d1d,0x998eac1b78861ce9ba700003b21582f3d31d4d9b12e1...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2023-12-02T07:21:27.507469,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
3,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x50f53f1b2900bbdb10fa54e29817eb0a12f21739d8c3...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2024-01-26T08:08:15.693777,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN
4,sale,,ethereum,,1701378179,1,0x29469395eaf6f95920e59f858042f0e28d98a20b,0x458dbf62e68463fe0b14c0f8dd69e695d23ca0cb,0x157175e7dee4ed8269415be4cf7c52fbbaa3eb113656...,1701378179,...,https://opensea.io/assets/ethereum/0xbc4ca0eda...,2024-01-26T17:09:27.131954,False,False,28300000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18.0,WETH,NaN,NaN


### 4. Number of unique NFTs

In [8]:
df["nft.metadata_url"].nunique(), df["nft.identifier"].nunique() # Only urls are truly unique 30860 is the correct number

(30860, 15644)

### 5. Extracting traits from metadata_url for every transaction

In [ ]:
# Get DataFrame with added nft.traits column
df = get_nft_traits(df)

In [ ]:
df.head()

### 6. Getting traits statistics for a collection

In [3]:
for collection in collection_list:
    get_traits_collection(collection)

In [4]:
with open('Collection_traits/bored-ape-kennel-club_traits.json') as z:
    traits_bakc = json.load(z)

In [21]:
traits_bakc.keys()

dict_keys(['categories', 'counts'])

In [22]:
traits_bakc["counts"]

{'Background': {'Outhouse': 2413,
  'SS Monke Biz': 2337,
  'Swamp': 2426,
  'Club Exterior': 1640,
  'Bar': 786},
 'Fur': {'Radioactive': 188,
  'Brown': 1522,
  'Tan': 2325,
  'Robot': 383,
  'Black': 1568,
  'Zombie': 390,
  'Noise': 181,
  'Pink': 1008,
  'Death Bot': 261,
  'Galaxy': 221,
  'Mega Death Bot': 171,
  'Cheetah': 611,
  'Cyberpunk': 271,
  'Trippy': 110,
  'DMT': 294,
  'Solid Gold': 98},
 'Neck': {'Japan Bandana': 110,
  'Christmas Wreath': 181,
  'Plaid Bowtie': 197,
  'BAYC Collar': 907,
  'Glow Necklace': 168,
  'Red BAYC Bandana': 55,
  'Whiskey Barrel': 84,
  'Camo Bandana': 117,
  'Pink Collar': 281,
  'Spiked Collar': 134,
  'Pinstripe Bandana': 180,
  'Red Collar With Bone': 260,
  'Life Preserver': 143,
  'Ribbon': 165,
  'Black BAYC Bandana': 129,
  'Hawaiian Bandana': 221,
  'Beer Can Necklace': 199,
  'Lei': 85,
  'Golden Bone Necklace': 197,
  'Flamingo Bandana': 114,
  'Gold Links': 178,
  'USA Bandana': 97,
  'Japanese Bandana': 92},
 'Mouth': {'Hot Do

In [25]:
df_traits = pd.DataFrame(traits_bakc["counts"])

In [26]:
df_traits.head()

,Background,Fur,Neck,Mouth,Eyes,Head,Feet,Back
Outhouse,2413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SS Monke Biz,2337.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Swamp,2426.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Club Exterior,1640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bar,786.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Blue Pack,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.0
Ninja Kunai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.0
Mega Rocket Launcher,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118.0
Toilet Paper Dispenser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0
